# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

Начнем анализ с открытия файла и изучения общей информации - тип строк, количество, наименование столбцов и тд методом .info()
Далее откроем первые 15 строчек файла для более ясной картины.

In [84]:
import pandas as pd #импртируем библиотеку pandas

try:
        df = pd.read_csv('C:/Users/datasets/data.csv')
except:
        df = pd.read_csv('/datasets/data.csv')

#df = pd.read_csv('/datasets/data.csv') - запасной вариант

df.info() 


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Посмотрим первые 15 строчек таблицы и проанализируем их

In [85]:
display(df.head(15))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Выведем названия столбцов - посмотрим все ли там верно

In [86]:
df.columns

Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')

**Вывод**



Общий вывод по таблице:
1) Названия столбцов логичные и правильные (на англ., со змеиным регистром, без пробелов и заглавных букв)

2) Столбец days_employed имеет непонятные данные (отрицательные значения) 

3) Есть пропуски в 2 столбцах - days_employed и total_income

4) Количество пропусков в столбцах days_employed и total_income идентично

5) В столбце образование - дубликаты, которые пишут по разному


## Шаг 2. Предобработка данных

### Обработка пропусков

Посмотрим какие есть пропуски:
    
1) Какое их количество

2) В каких они столбцах

Далее решим как и чем их заменить, или удалить


Алгоритм моего исследования:

In [87]:
#Поищем сумму всех пропусков в таблице
display(df.isna().sum()) # в 2 столбцах одинаковое количество пропусков в одинаковых строчках
# Узнаем процентное соотношение строк с пропусками от общего количества строк в подтверждение гипотезы выше
display(df.isna().mean())

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

children            0.000000
days_employed       0.100999
dob_years           0.000000
education           0.000000
education_id        0.000000
family_status       0.000000
family_status_id    0.000000
gender              0.000000
income_type         0.000000
debt                0.000000
total_income        0.100999
purpose             0.000000
dtype: float64

Выведем первые 10 строк из столбца с доходом - посмотрим на закономерности в пропусках

In [88]:
display(df[df['total_income'].isna()].head(10)) 

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,NaN,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,NaN,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,NaN,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,NaN,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


Пропуски в одинаковых строчках - люди не указывают доход и общий стаж не просто так.
Так как зарплата нужна будет для дальнейшего исследования - просто удалить эти строки не выход, нужно посчитать среднее и
медианное значение и заполнить им пропуски

In [89]:
print('Максимальное значение заработка в месяц', df['total_income'].max())
print('Минимальное значение заработка в месяц', df['total_income'].min())
median_zp = df['total_income'].median()
mean_zp = df['total_income'].mean()
print('Медианное значение заработка в месяц', median_zp)
print('Среднее значение заработка в месяц', mean_zp)

Максимальное значение заработка в месяц 2265604.028722744
Минимальное значение заработка в месяц 20667.26379327158
Медианное значение заработка в месяц 145017.93753253992
Среднее значение заработка в месяц 167422.30220817294


Медианное значение получилось меньше, чем среднее. Сравнив медианное и среднее значение, принимаю решение заменить пропуски 
медианным значением, так как разница между максимальным и минимальным значением больше 2,2 млн условных единиц, делая среднее значение выше.

In [90]:
# Заполним пропуски в значениях столбца ежемесячного дохода - методом fillna()
df['total_income'] = df['total_income'].fillna(median_zp)

Далее разберемся с пропуском в столбце со стажем работы.
Для исследования он не нужен - для этого заменим все значения на -1.

In [91]:
df['days_employed'] = df['days_employed'].fillna(-1)

In [92]:
# В конце удостоверимся что пустых строк нет, применив info().
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**


Произвели замену данных в столбцах - общий стаж работы и ежемесячный доход. Столбец ежемесячный доход нам далее необходим будет для исследования - просто удалить его мы не можем (доля пропусков составляла 10% - достаточно большое количество). Пропуски были у одних и тех же людей в столбцах стаж и доход - можно предположить что данные заполнили некорректно, или люди скрыли свой доход. 

### Замена типа данных

Далее произведем замену типов данных в столбцах - total_income и days_employed с float64 на int64 для удобства подсчетов данных,
визуально более приятного вида таблицы и правильности значений

In [93]:
# Заменим тип данных с помощью astype('int')
df['total_income'] = df['total_income'].astype('int')
df['days_employed'] = df['days_employed'].astype('int')
# Проверим - изменился ли тип данных в столбцах
df.info()




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


**Вывод**

Произвел замену типа данных с float на int - для более правильного отображения информации в столбцах и визуального удобства в таблице

### Обработка дубликатов

Алгоритм данной ячейки следующий - сначала посмотрим явные дубликаты и удалим их. Потом проверим неявные дубликаты и разберемся с ними

In [94]:
# Посчитаем сумму всех дубликатов в таблице
print('Общее количество дубликатов в таблице -',df.duplicated().sum())

Общее количество дубликатов в таблице - 54


Количество дубликатов 54 штуки - в принципе это означает что у нас 54 строки повторяются дважды, мы не знаем одинаковые это люди или нет - но полное совпадение странно, перезапишем датафрейм удалив дубликаты

In [95]:
df1 = df.drop_duplicates().reset_index(drop = True) 
      
print('Общее количество дубликатов в таблице -',df1.duplicated().sum())      

Общее количество дубликатов в таблице - 0


Дубликаты удалены

В столбацх у нас случаются дубликаты - в столбце с количеством детей и в столбце с уровнем образования - проверим данную гипотезу методом Unique()

In [96]:
print('Проверяем уникальные значения столбца children')
display(df1['children'].unique())
print('Видим некоторые странные значения - возможно (-1) это 1, а 20 - 2')

Проверяем уникальные значения столбца children


array([ 1,  0,  3,  2, -1,  4, 20,  5])

Видим некоторые странные значения - возможно (-1) это 1, а 20 - 2


По конверсии и логике людей с большим количеством детей должно быть меньше и меньше, но в данном случае людей у кого 20 детей 
больше, чем у кого 4 и 5 вместе взятых
Количество детей также не может быть отрицательным

In [97]:
print('Посмотрим количество уникальных значений и их количество для подтверждения теории')
display(df1['children'].value_counts())
print('Теория подтвердилась - это дубликаты в данных')

Посмотрим количество уникальных значений и их количество для подтверждения теории


 0     14107
 1      4809
 2      2052
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Теория подтвердилась - это дубликаты в данных


Заменим неверные значения методом replace и посмотрим на обновленный результат

In [98]:
df1['children'] = df1['children'].replace(-1, 1)
df1['children'] = df1['children'].replace(20, 2)
print('Поменяем данные в столбце с количеством детей на корректные значения и снова выведем на экран данный столбец')
display(df1['children'].value_counts())
# Отсортированная таблица с количеством детей у заемщиков выглядит корректно

Поменяем данные в столбце с количеством детей на корректные значения и снова выведем на экран данный столбец


0    14107
1     4856
2     2128
3      330
4       41
5        9
Name: children, dtype: int64

Отсортированная таблица с количеством детей у заемщиков выглядит корректно

Приступим к удалению дубликатов из столбца про образование

In [99]:
print('Далее разберемся с дубликатами в столбце образование и для начала найдем уникальные значения столбца')
display(df1['education'].unique())
print('Как мы видим все они корректные - но их необходимо привести к нижнему регистру - делаем это методом str.lower')
df1['education'] = df1['education'].str.lower()
print('Смотрим на результат - просим перечислить уникальные значения столбца')
display(df1['education'].unique())

Далее разберемся с дубликатами в столбце образование и для начала найдем уникальные значения столбца


array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

Как мы видим все они корректные - но их необходимо привести к нижнему регистру - делаем это методом str.lower
Смотрим на результат - просим перечислить уникальные значения столбца


array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

**Вывод**

Нашли и удалили явные дубликаты методом drop_duplicates().
Убрали дубликаты из 2 столбцов необходимых для исследования - количество детей и образование. Применили методы replace() и str.lower()

### Лемматизация

Импортируем библиотеку для лемматизации столбца - цели кредита
Там видны 4 явных цели, но они написаны по-разному - свадьба, недвижимость, автомобиль, образование

In [100]:
# импортируем библиотеку для лемматизации
from pymystem3 import Mystem
m = Mystem()
# Добавим новый столбец - purpose_id для лемм. Саму лемматизацию произведем методом apply(m.lemmatize)
df1['purpose_id'] = df1['purpose'].apply(m.lemmatize)

Отобразим новый столбец - для подтверждения успешности лемматизации

In [101]:
display(df1['purpose_id'].value_counts())


[автомобиль, \n]                                          972
[свадьба, \n]                                             793
[на,  , проведение,  , свадьба, \n]                       773
[сыграть,  , свадьба, \n]                                 769
[операция,  , с,  , недвижимость, \n]                     675
[покупка,  , коммерческий,  , недвижимость, \n]           662
[операция,  , с,  , жилье, \n]                            652
[покупка,  , жилье,  , для,  , сдача, \n]                 652
[операция,  , с,  , коммерческий,  , недвижимость, \n]    650
[покупка,  , жилье, \n]                                   646
[жилье, \n]                                               646
[покупка,  , жилье,  , для,  , семья, \n]                 638
[строительство,  , собственный,  , недвижимость, \n]      635
[недвижимость, \n]                                        633
[операция,  , со,  , свой,  , недвижимость, \n]           627
[строительство,  , жилой,  , недвижимость, \n]            625
[покупка

Cоздадим список лемм, для дальнейшей категоризации - тут хранится список всех лемм

In [102]:
lemmas = []
for i in df1['purpose_id']:
    lemmas.extend(i)
    
#display(lemmas) 
from collections import Counter
print(Counter(lemmas)) # посчитаем количество лемм по списку lemmas
# данный код отображает список всех лемм.
# теперь надо категоризировать данные - чтобы посчитать задание по целям кредита

Counter({' ': 33596, '\n': 21471, 'недвижимость': 6353, 'покупка': 5900, 'жилье': 4461, 'автомобиль': 4308, 'образование': 4014, 'с': 2918, 'операция': 2604, 'свадьба': 2335, 'свой': 2231, 'на': 2228, 'строительство': 1879, 'высокий': 1374, 'получение': 1315, 'коммерческий': 1312, 'для': 1290, 'жилой': 1231, 'сделка': 941, 'дополнительный': 907, 'заниматься': 904, 'проведение': 773, 'сыграть': 769, 'сдача': 652, 'семья': 638, 'собственный': 635, 'со': 627, 'ремонт': 607, 'подержанный': 486, 'подержать': 478, 'приобретение': 461, 'профильный': 436})


**Вывод**

Провел лемматизацию столбца с целями кредита - создав новый столбец с леммами. Далее сделал цикл, который добавляет каждое лемматизированное значение столбца в новый список и посчтиал количество лемм методом Count


### Категоризация данных

Создадим функцию, в которой категоризируем данные по лемматизированному столбцу. Далее применим эту метод apply к новому столбцу и посмотрим на результат

In [103]:
#display(lemmas)

def purpose_final(purpose):
    
    
    
    if 'свадьба' in purpose:
        return 'свадьба'
    elif 'недвижимость' in purpose:
        return 'недвижимость'
    elif 'автомобиль' in purpose:
        return 'автомобиль'
    elif 'жилье' in purpose:
        return 'недвижимость'
    else:
        return 'образование'
    
df1['purpose_final'] = df1['purpose_id'].apply(purpose_final)

display(df1.head(10))



    


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_id,purpose_final
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]",недвижимость
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]",автомобиль
2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]",недвижимость
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]",образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]",свадьба
5,0,-926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,"[покупка, , жилье, \n]",недвижимость
6,0,-2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,"[операция, , с, , жилье, \n]",недвижимость
7,0,-152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,"[образование, \n]",образование
8,2,-6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,"[на, , проведение, , свадьба, \n]",свадьба
9,0,-2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,"[покупка, , жилье, , для, , семья, \n]",недвижимость


**Вывод**

В результате лемматизации и категоризации создал новый столбец - с понятными целями взятия кредита и с ним можно работать в дальнейшем анализе

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Начнем отвечать на вопросы по исследованию и для начала рассчитаем общую конверсию

In [104]:
# Сперва рассчитаем конверсию по общему количеству людей с просрочками платежей
obch = df1['debt'].count()
dolz = df1[df1['debt'] == 1]['debt'].sum()
print(obch)
print(dolz)
percent = dolz / obch
print(percent)


21471
1741
0.08108611615667645


Общая конверсия равна - 0,081 (около 8% процентов). От этого показателя будем делать последующий анализ

Сделаем функцию, которая категоризирует людей на имеющих детей и без детей - добавим новый столбец методом apply и выведем его на экран

In [105]:
def children_final(purpose):
    
    if purpose >= 1:
        return 'есть дети'
    else:
        return 'нет детей'
    
df1['children_final'] = df1['children'].apply(children_final)

display(df1.head())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_id,purpose_final,children_final
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]",недвижимость,есть дети
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]",автомобиль,есть дети
2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]",недвижимость,нет детей
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]",образование,есть дети
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]",свадьба,нет детей


Дальше посчитаем количество людей с детьми и без, добавив столбец с задолженность и его аргументами - среднее и количество

In [106]:
t = df1.groupby('children_final')['debt'].agg(['mean','count'])
t.sort_values('count')
display(t)

,mean,count
children_final,,
есть дети,0.092070,7364
нет детей,0.075353,14107


**Вывод**

Есть зависимость в своевременном ежемесячном платеже и наличию детей в семье. Разница в среднем в 1,7 процента. 

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

Разберемся со следующим исследованием - выведем на экран по примеру предыдущей задачи - используем столбец family_status и debt

In [107]:
# Создадим новую таблицу, где будем учитывать данные по таблице family_status и debt и выделим среднюю конверсию по задолжностям
r = df1.groupby('family_status').agg({'debt':['count', 'mean']})
display(r)

debt          
                       count      mean
family_status                         
Не женат / не замужем   2810  0.097509
в разводе               1195  0.071130
вдовец / вдова           959  0.065693
гражданский брак        4163  0.093202
женат / замужем        12344  0.075421

**Вывод**

По данному исследованию можно сделать вывод, что семейное положение влияет на способность возвращать кредит без просрочек.
Самые дисциплинированные люди - вдовцы и вдовы - их конверсия задолженности - 6,5% (тогда как средняя задолженность 8%)
Люди женатые или в разводе укладываются в цифры ниже общей конверсии (7% и 7,5% соответсвенно)
Люди живущие в гражданском браке и не женатые - имеют самый больший процент просрочек от общего числа (9,2% и 9,7% соответсвенно).


- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Чтобы посчитать людей по уровню дохода - применим метод cut и qcut. Наши данные поделят примерно на равные по количеству людей части, отсюда сделаем вывод

In [108]:
df1['total_income_cut'] = pd.cut(df1['total_income'],4)
#display(df1['total_income_cut'])
#display(df1.groupby('total_income_cut')['debt'].agg(['count','mean']))
df1['total_income_cut1'] = pd.qcut(df1['total_income'],4,duplicates='drop')
display(df1.groupby('total_income_cut1')['debt'].agg(['count', 'mean']))

#display(df1['total_income_cut1'].value_counts())

,count,mean
total_income_cut1,,
"(20666.999, 107654.5]",5368,0.079545
"(107654.5, 145017.0]",6428,0.085096
"(145017.0, 195751.5]",4307,0.089157
"(195751.5, 2265604.0]",5368,0.071349


И второй способ - более понятный мне. Через функцию и категоризацию

In [109]:
def income_final(purpose):
    
    if purpose <= 100000:
        return 'низкий доход'
    if purpose <= 145000:
        return 'средний доход'
    if purpose <= 195000:
        return 'выше среднего доход'
    else:
        return 'высокий доход'
    
df1['total_income_final'] = df1['total_income'].apply(income_final)
display(df1.groupby('total_income_final')['debt'].agg(['count', 'mean']))

,count,mean
total_income_final,,
высокий доход,5423,0.071732
выше среднего доход,6374,0.085974
низкий доход,4463,0.079319
средний доход,5211,0.086356


**Вывод**

Разделив месячный доход на 4 равные части - мы пришли к выводу, количество людей примерно одинаковое.

Можно сделать вывод - люди с самым высоким доходом менее подвержены просрочкам, а люди в среднем сегменте больше. Прямая зависмимость просматривается

- Как разные цели кредита влияют на его возврат в срок?

Сделаем исследование по лемматизированному и категоризированному столбцу, добавив значение с просрочками по кредиту

In [110]:
display(df1.groupby('purpose_final')['debt'].agg(['count', 'mean']))

,count,mean
purpose_final,,
автомобиль,4308,0.093547
недвижимость,10814,0.072314
образование,4014,0.092177
свадьба,2335,0.079657


**Вывод**

Сравнивая 2 столбца - мы пришли к выводу, что люди, берущие кредит на свадьбу и недвижимость гораздо дисциплинированее в оплате, чем люди которые бурет деньги на образование и автомобиль - разница больше 1%

## Шаг 4. Общий вывод

Проведя полное исследование - мы удалили пропуски, дубликаты из таблицы, проанализировали данные для банка. Нашли закономерности при анализе - количество детей влияет на возврат средств, также влияет на возврат средств семейное положение. Также выяснили, что чем выше ежемесячный доход - тем легче не просрочить платеж. По целяи кредита, большие шаги (свадьба) и серьезные покупки (недвижимость) с большей вероятностью будут оплачены банку вовремя.